Route changed e nossa target

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [31]:
df_train=pd.read_csv('./data/clean_train.csv', index_col=None)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199652 entries, 0 to 199651
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   tr_id                   199652 non-null  float64
 1   tr_dst                  199652 non-null  float64
 2   total_probes_sent       199652 non-null  float64
 3   total_replies_last_hop  199652 non-null  float64
 4   route_changed           199652 non-null  float64
 5   seconds_since_start     199652 non-null  float64
 6   mean_rtt                199652 non-null  float64
dtypes: float64(7)
memory usage: 10.7 MB


In [32]:
df_test=pd.read_csv('./data/clean_test.csv', index_col=None)
df_test=df_test.sample(frac=0.1,random_state=12345)

In [33]:
X=df_train.drop('route_changed',axis=1)
y=df_train['route_changed']

seed_list=np.random.choice(range(1, 1000000), size=100, replace=False)

In [34]:
print(X.shape)
print(y.shape)

(199652, 6)
(199652,)


In [35]:
def K_Fold_XGBoost(X, y,n_splits=10,random_state=12345,n_estimators=50, n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= xgb.XGBClassifier(n_estimators=n_estimators, max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

#def K_Fold_Forest(X, y,n_splits=10,random_state=12345,n_estimators=20,n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= RandomForestClassifier(n_estimators=n_estimators,max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Tree(X, y,n_splits=10,random_state=12345,n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= DecisionTreeClassifier(max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Logistic(X, y,n_splits=10,random_state=12345):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= LogisticRegression(random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

In [36]:
K_Fold_XGBoost(X,y,10,12345,21,None)

[np.float64(0.982003686758348),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [37]:
K_Fold_Logistic(X,y)

[np.float64(0.982003686758348),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [38]:
results=K_Fold_Tree(X,y)

In [39]:
print(results)

[np.float64(0.9814927979618945), np.float64(0.07195701357466064), np.float64(0.002225317239244816), np.float64(0.0043057800011353144)]


In [40]:
def data_collection_tree(seed_list,X,y,n=30):
    df=pd.DataFrame()
    for i in range(1,n+1):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_Tree(X,y,10,seed,i)
            result_list.append(result)
        df[i]=result_list
        results_tree_df=pd.DataFrame(result_list)
        results_tree_df.to_csv('./data/result_tree_'+ str(i) +'.csv',index=False)
        print(i)
    print('Done')
    return df

def data_collection_xgboost(seed_list,X,y,n=61):
    df=pd.DataFrame()
    for i in range(1,n+1,5):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_XGBoost(X,y,10,seed,i,None)
            result_list.append(result)
        df[i]=result_list
        print(i)
        results_xgboost_df=pd.DataFrame(result_list)
        results_xgboost_df.to_csv('./data/result_xgboost_'+ str(i) +'.csv',index=False)
    print('Done')
    return df

def data_collection_logistic(seed_list, X, y):
    result_list=[]
    for seed in seed_list:
        result=K_Fold_Logistic(X,y,10,seed)
        result_list.append(result)
    print('Done')
    return result_list

In [41]:
results_tree=data_collection_tree(seed_list,X,y)
results_tree_df=pd.DataFrame(results_tree, columns=['acc','precision','recall','F1'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Done


In [42]:
results_tree_df.to_csv('./data/result_tree.csv',index=False)

In [43]:
results_xgboost=data_collection_xgboost(seed_list,X,y)

1
6
11
16
21
26
31
36
41
46
51
56
61
Done


In [44]:
#for i in range(1,41,5):
#    temp=list(results_xgboost[[i]].values)
#    print(temp)
#    results_xgboost_df=pd.DataFrame(temp)
#    results_xgboost_df.to_csv('./data/result_xgboost_'+ str(i) +'.csv',index=False)


In [45]:
#for i in range(1,21):
#    temp=results_tree[[i]]
#    print(temp)
#    results_xgboost_df=pd.DataFrame(temp)
#    results_xgboost_df.to_csv('./data/result_tree_'+ str(i) +'.csv',index=False)

In [46]:
results_logistic=data_collection_logistic(seed_list,X,y)
results_logistic_df=pd.DataFrame(results_logistic, columns=['acc','precision','recall','F1'])

Done


In [47]:
results_logistic_df.to_csv('./data/result_logistic.csv',index=False)

In [48]:
y.sum()

np.float64(3593.0)

In [49]:
len(y)

199652

In [50]:
print(results_tree)

                                    1                                   2   \
0   [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
1   [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
2   [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
3   [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
4   [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
..                                 ...                                 ...   
95  [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
96  [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
97  [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
98  [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   
99  [0.982003686758348, 0.0, 0.0, 0.0]  [0.982003686758348, 0.0, 0.0, 0.0]   

                                                   3   \
0   [0

In [51]:
print(results_xgboost[1])

0     [0.982003686758348, 0.0, 0.0, 0.0]
1     [0.982003686758348, 0.0, 0.0, 0.0]
2     [0.982003686758348, 0.0, 0.0, 0.0]
3     [0.982003686758348, 0.0, 0.0, 0.0]
4     [0.982003686758348, 0.0, 0.0, 0.0]
                     ...                
95    [0.982003686758348, 0.0, 0.0, 0.0]
96    [0.982003686758348, 0.0, 0.0, 0.0]
97    [0.982003686758348, 0.0, 0.0, 0.0]
98    [0.982003686758348, 0.0, 0.0, 0.0]
99    [0.982003686758348, 0.0, 0.0, 0.0]
Name: 1, Length: 100, dtype: object


In [52]:
print(results_logistic)

[[np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float64(0.0)], [np.float64(0.982003686758348), np.float64(0.0), np.float64(0.0), np.float6

In [53]:
type(results_logistic)

list

In [54]:
type(results_xgboost.values)

numpy.ndarray

## Upsampling e downsampling os dados

In [55]:
positive=df_train.loc[df_train['route_changed']==True]
negative=df_train.loc[df_train['route_changed']==False]
print(len(negative),len(positive))

196059 3593


In [56]:
negative=negative.sample(frac=0.5)

In [57]:
list_df=[negative,positive,positive,positive,positive,positive]
sampled_train_df = pd.concat(list_df)

In [58]:
sampled_train_df=sampled_train_df.sample(frac=1).reset_index(drop=True)

In [59]:
len(sampled_train_df)

115995

In [60]:
alt_X=sampled_train_df.drop('route_changed',axis=1)
alt_y=sampled_train_df['route_changed']

In [61]:
def K_Fold_XGBoost_Alt(X, y, Alt_X, Alt_y,n_splits=10,random_state=12345,n_estimators=50, n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= xgb.XGBClassifier(n_estimators=n_estimators, max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    original_folds=[]
    for fold, (train, test) in enumerate(cv.split(X, y)):
        original_folds.append(test)
    
    for fold, (train, test) in enumerate(cv.split(Alt_X, Alt_y)):
        model.fit(Alt_X.iloc[train],Alt_y.iloc[train])
        prediction=model.predict(X.iloc[original_folds[fold]])
        acc_list.append(accuracy_score(y.iloc[original_folds[fold]],prediction))
        precision_list.append(precision_score(y.iloc[original_folds[fold]],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[original_folds[fold]],prediction))
        f1_list.append(f1_score(y.iloc[original_folds[fold]],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Tree_Alt(X, y, Alt_X, Alt_y,n_splits=10,random_state=12345,n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= DecisionTreeClassifier(max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    original_folds=[]
    for fold, (train, test) in enumerate(cv.split(X, y)):
        original_folds.append(test)
    
    for fold, (train, test) in enumerate(cv.split(Alt_X, Alt_y)):
        model.fit(Alt_X.iloc[train],Alt_y.iloc[train])
        prediction=model.predict(X.iloc[original_folds[fold]])
        acc_list.append(accuracy_score(y.iloc[original_folds[fold]],prediction))
        precision_list.append(precision_score(y.iloc[original_folds[fold]],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[original_folds[fold]],prediction))
        f1_list.append(f1_score(y.iloc[original_folds[fold]],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Logistic_Alt(X, y, Alt_X, Alt_y,n_splits=10,random_state=12345):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= LogisticRegression(random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    original_folds=[]
    for fold, (train, test) in enumerate(cv.split(X, y)):
        original_folds.append(test)
    
    for fold, (train, test) in enumerate(cv.split(Alt_X, Alt_y)):
        model.fit(Alt_X.iloc[train],Alt_y.iloc[train])
        prediction=model.predict(X.iloc[original_folds[fold]])
        acc_list.append(accuracy_score(y.iloc[original_folds[fold]],prediction))
        precision_list.append(precision_score(y.iloc[original_folds[fold]],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[original_folds[fold]],prediction))
        f1_list.append(f1_score(y.iloc[original_folds[fold]],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def data_collection_tree_alt(seed_list,X,y,Alt_X,Alt_y,n=30):
    df=pd.DataFrame()
    for i in range(1,n+1):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_Tree_Alt(X,y,Alt_X,Alt_y,10,seed,i)
            result_list.append(result)
        df[i]=result_list
        results_tree_df=pd.DataFrame(result_list)
        results_tree_df.to_csv('./data/result_tree_sampled_'+ str(i) +'.csv',index=False)
        print(i)
    print('Done')
    return df

def data_collection_xgboost_alt(seed_list,X,y,Alt_X,Alt_y,n=61):
    df=pd.DataFrame()
    for i in range(1,n+1,5):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_XGBoost_Alt(X,y,Alt_X,Alt_y,10,seed,i,None)
            result_list.append(result)
        df[i]=result_list
        print(i)
        results_xgboost_df=pd.DataFrame(result_list)
        results_xgboost_df.to_csv('./data/result_xgboost_sampled_'+ str(i) +'.csv',index=False)
    print('Done')
    return df


def data_collection_logistic_alt(seed_list, X, y,Alt_X,Alt_y):
    result_list=[]
    for seed in seed_list:
        result=K_Fold_Logistic_Alt(X,y,Alt_X,Alt_y,10,seed)
        result_list.append(result)
    print('Done')
    return result_list

In [62]:
results_xgboost_sampled=data_collection_xgboost_alt(seed_list,X,y,alt_X,alt_y)

1
6
11
16
21
26
31
36
41
46
51
56
61
Done


In [ ]:
results_tree_sampled=data_collection_tree_alt(seed_list,X,y,alt_X,alt_y)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [ ]:
results_logistic_sampled=data_collection_logistic_alt(seed_list,X,y,alt_X,alt_y)
results_logistic_sampled_df=pd.DataFrame(results_logistic_sampled, columns=['acc','precision','recall','F1'])

Done


In [ ]:
results_logistic_sampled_df.to_csv('./data/result_logistic_sampled.csv',index=False)